In [1]:
# pulls from the FantasyPros and ESPN API

## **Important: The dataframes are built using csv files in the current working directory so 
# do not delete or comment out these functions or lines of code that create the csv files

In [1]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display
from datetime import datetime
import nfl_data_py as nfl
import os

In [3]:
# Define the template URLs for the QB, RB, and WR positions
espn_urls = {
    "QB": "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page={page}&limit=50&category=offense%3Apassing&sort=passing.passingYards%3Adesc&season={year}&seasontype={seasontype}",
    "RB": "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page={page}&limit=50&category=offense%3Arushing&sort=rushing.rushingYards%3Adesc&season={year}&seasontype={seasontype}",
    "WR": "https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/statistics/byathlete?region=us&lang=en&contentorigin=espn&isqualified=false&page={page}&limit=50&category=offense%3Areceiving&sort=receiving.receivingYards%3Adesc&season={year}&seasontype={seasontype}"
}

In [5]:
# **IMPORTANT: this function outputs the combined qb-betting lines df
# Get current year and week for NFL
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4) ## *** Reset the date at the start of the NFL season ***
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Set the current NFL year and week
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [7]:
# Adjust the fetch function to return both data and pagination info for verification
def fetch_position_data_with_verification(position, url_template):
    page = 1
    all_players = []
    total_pages = 1  # Default to 1 page unless pagination indicates more
    
    while True:
        # Construct the API URL for the current page
        url = url_template.format(page=page, year=current_year, seasontype=seasontype)
        response = requests.get(url)
        
        # If response is successful, process the data
        if response.status_code == 200:
            data = response.json()
            athletes = data.get('athletes', [])
            
            # Get pagination information for verification
            if page == 1:
                pagination = data.get('pagination', {})
                total_pages = pagination.get('pages', 1)  # Total number of pages
            
            if not athletes:
                break  # Stop if no more athletes are available
            
            for athlete_data in athletes:
                athlete = athlete_data['athlete']

                # Extract relevant data for the base columns (excluding stats)
                player_info = {
                    'year': current_year,
                    'week': current_week,
                    'player_id': athlete.get('id', 'N/A'),
                    'player': athlete.get('displayName', 'N/A'),
                    'position': position,
                    'team': athlete.get('teamShortName', 'N/A')
                }

                all_players.append(player_info)
            
            page += 1  # Increment to the next page
        else:
            break  # Stop if there's an error in fetching data
    
    return all_players, total_pages

In [9]:
# Function to convert fetched data into a DataFrame
def create_dataframe(position_data):
    return pd.DataFrame(position_data)
    
# Updated process function to include verification step
def process_and_verify_position_data():
    # Fetch data for each position and track total pages
    qb_data, qb_pages = fetch_position_data_with_verification("QB", espn_urls["QB"])
    rb_data, rb_pages = fetch_position_data_with_verification("RB", espn_urls["RB"])
    wr_data, wr_pages = fetch_position_data_with_verification("WR", espn_urls["WR"])
    
    # Convert fetched data into DataFrames
    df_qb = create_dataframe(qb_data)
    df_rb = create_dataframe(rb_data)
    df_wr = create_dataframe(wr_data)
    
    # Verification output
    print(f"QB: Fetched {len(df_qb)} rows across {qb_pages} pages.")
    print(f"RB: Fetched {len(df_rb)} rows across {rb_pages} pages.")
    print(f"WR: Fetched {len(df_wr)} rows across {wr_pages} pages.")
    
    # Display the first few rows for review
    display(df_qb.head())
    display(df_rb.head())
    display(df_wr.head())
    
    return df_qb, df_rb, df_wr

# Call the function to fetch, verify, and display the data
df_qb, df_rb, df_wr = process_and_verify_position_data()

QB: Fetched 65 rows across 2 pages.
RB: Fetched 233 rows across 5 pages.
WR: Fetched 390 rows across 8 pages.


,year,week,player_id,player,position,team
0,2024,7,15864,Geno Smith,QB,SEA
1,2024,7,4361741,Brock Purdy,QB,SF
2,2024,7,2577417,Dak Prescott,QB,DAL
3,2024,7,14880,Kirk Cousins,QB,ATL
4,2024,7,3915511,Joe Burrow,QB,CIN


,year,week,player_id,player,position,team
0,2024,7,3043078,Derrick Henry,RB,BAL
1,2024,7,4360569,Jordan Mason,RB,SF
2,2024,7,4241416,Chuba Hubbard,RB,CAR
3,2024,7,3929630,Saquon Barkley,RB,PHI
4,2024,7,4047365,Josh Jacobs,RB,GB


,year,week,player_id,player,position,team
0,2024,7,4258173,Nico Collins,WR,HOU
1,2024,7,4362628,Ja'Marr Chase,WR,CIN
2,2024,7,3116165,Chris Godwin,WR,TB
3,2024,7,4047650,DK Metcalf,WR,SEA
4,2024,7,4241389,CeeDee Lamb,WR,DAL


In [11]:
# Function to generate FantasyPros URLs based on the positions
# Function to generate FantasyPros URLs based on the positions
def generate_fantasy_pros_urls(season, positions=None, week=None, scoring=None):
    base_url = f"https://api.fantasypros.com/public/v2/json/nfl/{season}/projections"
    # If positions is not provided, default to QB, RB, WR. Otherwise, use the list directly.
    positions_list = ['QB', 'RB', 'WR'] if positions is None else positions  # Remove split
    scoring_str = scoring.replace("'", "") if scoring else None
    generated_urls = []

    for position in positions_list:
        params = {'position': position}
        if season:
            params['season'] = season
        if week:
            params['week'] = week
        if scoring:
            params['scoring'] = scoring_str
        query_string = requests.compat.urlencode(params)
        full_url = f"{base_url}?{query_string}"
        generated_urls.append(full_url)

    return generated_urls

# Function to fetch data from FantasyPros API
def fetch_data(url, headers=None):
    response = requests.get(url, headers=headers)
    try:
        response.raise_for_status()
        return response.json()  # Return the JSON data
    except requests.RequestException as e:
        print(f"Failed to retrieve {url}. Error: {e}")
        return None

# Function to fetch and handle FantasyPros data for given positions and stats
def fetch_fantasy_pros_data(season, positions=None, week=None, scoring=None):
    api_key = os.getenv('api_key')
    if not api_key:
        print("API key is not set.")
        return None
    
    headers = {'x-api-key': api_key}
    urls = generate_fantasy_pros_urls(season, positions, week, scoring)
    all_data = []
    
    for url in urls:
        print(f"Fetching FantasyPros data from: {url}")
        response = fetch_data(url, headers)
        if response and 'players' in response:
            players_data = response['players']
            for player in players_data:
                # Extract general columns
                player_info = {
                    'name': player['name'],
                    'points': player['stats'].get('points', 0),
                    'points_ppr': player['stats'].get('points_ppr', 0),
                    'points_half': player['stats'].get('points_half', 0)
                }
                # Extract position-specific columns based on position
                position = player.get('position_id')
                if position == 'QB':
                    player_info.update({
                        'passing_attempts': player['stats'].get('pass_att', 0),
                        'passing_completions': player['stats'].get('pass_cmp', 0),
                        'passing_yards': player['stats'].get('pass_yds', 0),
                        'passing_tds': player['stats'].get('pass_tds', 0)
                    })
                elif position == 'RB':
                    player_info.update({
                        'rushing_attempts': player['stats'].get('rush_att', 0),
                        'rushing_yards': player['stats'].get('rush_yds', 0),
                        'rushing_tds': player['stats'].get('rush_tds', 0),
                        'receptions': player['stats'].get('rec_rec', 0),
                        'reception_yards': player['stats'].get('rec_yds', 0),
                        'reception_tds': player['stats'].get('rec_tds', 0)
                    })
                elif position == 'WR':
                    player_info.update({
                        'receptions': player['stats'].get('rec_rec', 0),
                        'reception_yards': player['stats'].get('rec_yds', 0),
                        'reception_tds': player['stats'].get('rec_tds', 0)
                    })
                all_data.append(player_info)
    
    return pd.DataFrame(all_data)

In [13]:
# Function to merge ESPN data with FantasyPros data, keeping only the relevant columns for each position
def merge_espn_fantasypros(espn_df, fantasypros_df, position):
    # Extract relevant columns based on position
    if position == 'QB':
        # Extract only QB relevant columns
        fantasypros_df = fantasypros_df[['name', 'points', 'points_ppr', 'points_half', 
                                         'passing_attempts', 'passing_completions', 'passing_yards', 'passing_tds']]
    elif position == 'RB':
        # Extract only RB relevant columns
        fantasypros_df = fantasypros_df[['name', 'points', 'points_ppr', 'points_half', 
                                         'rushing_attempts', 'rushing_yards', 'rushing_tds', 
                                         'receptions', 'reception_yards', 'reception_tds']]
    elif position == 'WR':
        # Extract only WR relevant columns
        fantasypros_df = fantasypros_df[['name', 'points', 'points_ppr', 'points_half', 
                                         'receptions', 'reception_yards', 'reception_tds']]

    # Merge on 'player' from ESPN and 'name' from FantasyPros
    merged_df = pd.merge(espn_df, fantasypros_df, left_on='player', right_on='name', how='left')
    
    # Drop the redundant 'name' column from FantasyPros
    merged_df.drop(columns=['name'], inplace=True)
    
    return merged_df

In [34]:
# Function to fetch, merge, and save ESPN and FantasyPros data for all positions
def process_and_merge_fantasypros_data(df_qb, df_rb, df_wr, scoring='STD'):
    # Fetch current season and week dynamically
    current_week = get_current_week()
    season = datetime.now().year

    # Fetch FantasyPros data for all positions
    fantasypros_data = fetch_fantasy_pros_data(season=season, positions=['QB', 'RB', 'WR'], week=current_week, scoring=scoring)

    # Merging ESPN dataframes with FantasyPros data, keeping only relevant columns
    df_qb_merged = merge_espn_fantasypros(df_qb, fantasypros_data, 'QB')
    df_rb_merged = merge_espn_fantasypros(df_rb, fantasypros_data, 'RB')
    df_wr_merged = merge_espn_fantasypros(df_wr, fantasypros_data, 'WR')


    # Display the merged dataframes
    display(df_qb_merged.head())
    display(df_rb_merged.head())
    display(df_wr_merged.head())
    
    return df_qb_merged, df_rb_merged, df_wr_merged

# Call the function to fetch, merge, save, and display the data
df_qb_merged, df_rb_merged, df_wr_merged = process_and_merge_fantasypros_data(df_qb, df_rb, df_wr)


Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=QB&season=2024&week=7&scoring=STD
Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=RB&season=2024&week=7&scoring=STD
Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=WR&season=2024&week=7&scoring=STD


,year,week,player_id,player,position,team,points,points_ppr,points_half,passing_attempts,passing_completions,passing_yards,passing_tds
0,2024,7,15864,Geno Smith,QB,SEA,17.41,17.41,17.41,37.83,26.32,262.52,1.42
1,2024,7,4361741,Brock Purdy,QB,SF,17.68,17.68,17.68,31.92,20.88,253.64,1.63
2,2024,7,2577417,Dak Prescott,QB,DAL,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2024,7,14880,Kirk Cousins,QB,ATL,16.76,16.76,16.76,34.13,22.74,260.32,1.71
4,2024,7,3915511,Joe Burrow,QB,CIN,17.91,17.91,17.91,34.03,22.86,256.04,1.68


,year,week,player_id,player,position,team,points,points_ppr,points_half,rushing_attempts,rushing_yards,rushing_tds,receptions,reception_yards,reception_tds
0,2024,7,3043078,Derrick Henry,RB,BAL,15.73,16.96,16.34,19.31,91.51,0.90,1.22,10.17,0.05
1,2024,7,4360569,Jordan Mason,RB,SF,10.96,12.56,11.76,15.95,68.95,0.47,1.60,11.42,0.04
2,2024,7,4241416,Chuba Hubbard,RB,CAR,11.91,15.04,13.47,16.63,69.46,0.38,3.14,22.85,0.10
3,2024,7,3929630,Saquon Barkley,RB,PHI,14.49,17.25,15.87,19.14,85.08,0.56,2.76,20.09,0.14
4,2024,7,4047365,Josh Jacobs,RB,GB,11.00,13.44,12.22,15.52,65.18,0.40,2.44,16.91,0.09


,year,week,player_id,player,position,team,points,points_ppr,points_half,receptions,reception_yards,reception_tds
0,2024,7,4258173,Nico Collins,WR,HOU,NaN,NaN,NaN,NaN,NaN,NaN
1,2024,7,4362628,Ja'Marr Chase,WR,CIN,11.49,17.53,14.51,6.04,82.43,0.56
2,2024,7,3116165,Chris Godwin,WR,TB,10.60,17.17,13.89,6.57,77.93,0.47
3,2024,7,4047650,DK Metcalf,WR,SEA,9.92,15.20,12.56,5.28,72.42,0.47
4,2024,7,4241389,CeeDee Lamb,WR,DAL,0.00,0.00,0.00,0.00,0.00,0.00


In [37]:
def scrape_salary_changes():
    # URL of the FantasyPros salary changes page
    url = "https://www.fantasypros.com/daily-fantasy/nfl/fanduel-salary-changes.php"
    
    # Fetch the page content
    response = requests.get(url)
    
    # Check if the page was fetched successfully
    if response.status_code != 200:
        print(f"Failed to fetch the page. Status code: {response.status_code}")
        return
    
    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Locate the table containing the salary changes (assuming it's the first table)
    table = soup.find('table')  # Adjust if necessary based on the page structure
    
    # Extract the table headers
    headers = [header.text for header in table.find_all('th')]
    
    # Extract the table rows
    rows = []
    for row in table.find_all('tr')[1:]:  # Skip the header row
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]  # Clean up whitespace
        rows.append(cols)
    
    # Create a DataFrame with the scraped data
    salary_changes_df = pd.DataFrame(rows, columns=headers)
        
    # Display the first few rows of the DataFrame
    display(salary_changes_df.head())
    
    return salary_changes_df

# Run the function to scrape and display salary changes
scrape_salary_changes()

,ECR,Player,Kickoff,Opp,This Week,Last Week,Difference
0,-,Riley Sharp (BAL - TE),Mon 8:15PM,@TB,"$4,000","$4,000",0
1,249,John Metchie III (HOU - WR),Sun 1:00PM,@GB,"$4,400","$4,300",+100
2,-,Trey Knox (MIN - TE),Sun 1:00PM,DET,"$4,000",-,-
3,391,Kevin Harris (NE - RB),Sun 9:30AM,@JAC,"$4,000","$4,000",0
4,2,Jayden Daniels (WAS - QB),Sun 4:05PM,CAR,"$9,100","$8,700",+400


,ECR,Player,Kickoff,Opp,This Week,Last Week,Difference
0,-,Riley Sharp (BAL - TE),Mon 8:15PM,@TB,"$4,000","$4,000",0
1,249,John Metchie III (HOU - WR),Sun 1:00PM,@GB,"$4,400","$4,300",+100
2,-,Trey Knox (MIN - TE),Sun 1:00PM,DET,"$4,000",-,-
3,391,Kevin Harris (NE - RB),Sun 9:30AM,@JAC,"$4,000","$4,000",0
4,2,Jayden Daniels (WAS - QB),Sun 4:05PM,CAR,"$9,100","$8,700",+400
...,...,...,...,...,...,...,...
961,-,D.J. Montgomery (IND - WR),Sun 1:00PM,MIA,"$4,000","$4,000",0
962,-,Montrell Washington (KC - WR),Sun 4:25PM,@SF,"$4,000",-,-
963,-,Nick Muse (MIN - TE),Sun 1:00PM,DET,"$4,000",-,-
964,-,Owen Wright (BAL - RB),Mon 8:15PM,@TB,"$4,000","$4,000",0


In [40]:
def fetch_salary_changes():
    # Directly scrape and return the salary changes DataFrame
    print("Fetching salary changes data by scraping...")
    salary_changes_df = scrape_salary_changes()
    return salary_changes_df

In [43]:
def process_salary_changes_by_position():
    # Step 1: Fetch the salary changes data in memory
    salary_changes_df = fetch_salary_changes()
    
    # Step 2: Split 'Player' column into three separate columns (FirstName, LastName, Team-Position)
    salary_changes_df[['FirstName', 'LastName', 'Team-Position']] = salary_changes_df['Player'].str.extract(r'(\w+)\s+(\w+)\s+\((.*)\)')
    
    # Step 3: Drop the original 'Player' column and the first column (Rankings)
    salary_changes_df.drop(columns=['Player', salary_changes_df.columns[0]], inplace=True)
    
    # Step 4: Split 'Team-Position' into separate 'Team' and 'Position' columns
    salary_changes_df[['Team', 'Position']] = salary_changes_df['Team-Position'].str.extract(r'(\w+)\s*-\s*(\w+)')
    
    # Step 5: Drop 'Team-Position' and any unwanted columns like 'Kickoff' and 'Opp'
    salary_changes_df.drop(columns=['Team-Position', 'Kickoff', 'Opp'], inplace=True, errors='ignore')
    
    # Step 6: Reorder the columns to match the desired order
    salary_changes_df = salary_changes_df[['FirstName', 'LastName', 'Team', 'Position', 'This Week', 'Last Week', 'Difference']]
    
    # Step 7: Split the DataFrame by position
    df_qb = salary_changes_df[salary_changes_df['Position'] == 'QB']
    df_wr = salary_changes_df[salary_changes_df['Position'] == 'WR']
    df_rb = salary_changes_df[salary_changes_df['Position'] == 'RB']
    
    # Display the first few rows of each DataFrame for verification
    display(df_qb.head())
    display(df_wr.head())
    display(df_rb.head())
    
    return df_qb, df_wr, df_rb

# Call the function to process the salary changes by position
df_qb_salary, df_wr_salary, df_rb_salary = process_salary_changes_by_position()


Fetching salary changes data by scraping...


,ECR,Player,Kickoff,Opp,This Week,Last Week,Difference
0,-,Riley Sharp (BAL - TE),Mon 8:15PM,@TB,"$4,000","$4,000",0
1,249,John Metchie III (HOU - WR),Sun 1:00PM,@GB,"$4,400","$4,300",+100
2,-,Trey Knox (MIN - TE),Sun 1:00PM,DET,"$4,000",-,-
3,391,Kevin Harris (NE - RB),Sun 9:30AM,@JAC,"$4,000","$4,000",0
4,2,Jayden Daniels (WAS - QB),Sun 4:05PM,CAR,"$9,100","$8,700",+400


,FirstName,LastName,Team,Position,This Week,Last Week,Difference
4,Jayden,Daniels,WAS,QB,"$9,100","$8,700",+400
13,Kedon,Slovis,HOU,QB,"$6,000","$6,000",0
15,Bo,Nix,DEN,QB,"$6,800","$6,700",+100
21,Max,Duggan,LAC,QB,"$6,000","$6,000",0
41,Easton,Stick,LAC,QB,"$6,000","$6,000",0


,FirstName,LastName,Team,Position,This Week,Last Week,Difference
1,Metchie,III,HOU,WR,"$4,400","$4,300",+100
5,Ricky,Pearsall,SF,WR,"$4,800","$4,000",+800
7,Joseph,Ngata,PHI,WR,"$4,000","$4,000",0
8,Anthony,Gould,IND,WR,"$4,000","$4,000",0
10,Trey,Palmer,TB,WR,"$4,100","$4,200",-100


,FirstName,LastName,Team,Position,This Week,Last Week,Difference
3,Kevin,Harris,NE,RB,"$4,000","$4,000",0
12,Eric,Gray,NYG,RB,"$5,000","$4,700",+300
16,Austin,Jones,WAS,RB,"$4,000","$4,000",0
19,Breece,Hall,NYJ,RB,"$7,600","$8,000",-400
25,Aidan,Robbins,CLE,RB,"$4,000","$4,000",0


In [59]:
# Function renamed to merge_salary_and_projections
def merge_salary_and_projections(df_qb_salary, df_rb_salary, df_wr_salary, df_qb_proj, df_rb_proj, df_wr_proj):
    """
    This function takes the salary DataFrames and the projection DataFrames 
    and merges them based on player names for QB, RB, and WR positions.
    """

    # Split 'player' column into 'FirstName' and 'LastName' in projection DataFrames
    df_qb_proj[['FirstName', 'LastName']] = df_qb_proj['player'].str.split(' ', n=1, expand=True)
    df_rb_proj[['FirstName', 'LastName']] = df_rb_proj['player'].str.split(' ', n=1, expand=True)
    df_wr_proj[['FirstName', 'LastName']] = df_wr_proj['player'].str.split(' ', n=1, expand=True)

    # Merge salary and projections for QB
    df_qb_merge_projection_salary = pd.merge(df_qb_proj, df_qb_salary, how='left', left_on=['FirstName', 'LastName'], right_on=['FirstName', 'LastName'])
    print(f"QB merged data: {df_qb_merge_projection_salary.shape[0]} rows")

    # Merge salary and projections for RB
    df_rb_merge_projection_salary = pd.merge(df_rb_proj, df_rb_salary, how='left', left_on=['FirstName', 'LastName'], right_on=['FirstName', 'LastName'])
    print(f"RB merged data: {df_rb_merge_projection_salary.shape[0]} rows")

    # Merge salary and projections for WR
    df_wr_merge_projection_salary = pd.merge(df_wr_proj, df_wr_salary, how='left', left_on=['FirstName', 'LastName'], right_on=['FirstName', 'LastName'])
    print(f"WR merged data: {df_wr_merge_projection_salary.shape[0]} rows")

    # Display the first few rows of each merged DataFrame for verification
    display(df_qb_merge_projection_salary.head())
    display(df_rb_merge_projection_salary.head())
    display(df_wr_merge_projection_salary.head())

    return df_qb_merge_projection_salary, df_rb_merge_projection_salary, df_wr_merge_projection_salary


# Fetch, merge, and return projections from ESPN and FantasyPros (already implemented)
df_qb_proj, df_rb_proj, df_wr_proj = process_and_merge_fantasypros_data(df_qb, df_rb, df_wr, scoring='STD')

# Now, merge these with salary data
df_qb_merge_projection_salary, df_rb_merge_projection_salary, df_wr_merge_projection_salary = merge_salary_and_projections(
    df_qb_salary, df_rb_salary, df_wr_salary,  # Salary DataFrames
    df_qb_proj, df_rb_proj, df_wr_proj  # Projection DataFrames from ESPN and FantasyPros
)


Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=QB&season=2024&week=7&scoring=STD
Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=RB&season=2024&week=7&scoring=STD
Fetching FantasyPros data from: https://api.fantasypros.com/public/v2/json/nfl/2024/projections?position=WR&season=2024&week=7&scoring=STD


,year,week,player_id,player,position,team,points,points_ppr,points_half,passing_attempts,passing_completions,passing_yards,passing_tds
0,2024,7,15864,Geno Smith,QB,SEA,17.41,17.41,17.41,37.83,26.32,262.52,1.42
1,2024,7,4361741,Brock Purdy,QB,SF,17.68,17.68,17.68,31.92,20.88,253.64,1.63
2,2024,7,2577417,Dak Prescott,QB,DAL,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2024,7,14880,Kirk Cousins,QB,ATL,16.76,16.76,16.76,34.13,22.74,260.32,1.71
4,2024,7,3915511,Joe Burrow,QB,CIN,17.91,17.91,17.91,34.03,22.86,256.04,1.68


,year,week,player_id,player,position,team,points,points_ppr,points_half,rushing_attempts,rushing_yards,rushing_tds,receptions,reception_yards,reception_tds
0,2024,7,3043078,Derrick Henry,RB,BAL,15.73,16.96,16.34,19.31,91.51,0.90,1.22,10.17,0.05
1,2024,7,4360569,Jordan Mason,RB,SF,10.96,12.56,11.76,15.95,68.95,0.47,1.60,11.42,0.04
2,2024,7,4241416,Chuba Hubbard,RB,CAR,11.91,15.04,13.47,16.63,69.46,0.38,3.14,22.85,0.10
3,2024,7,3929630,Saquon Barkley,RB,PHI,14.49,17.25,15.87,19.14,85.08,0.56,2.76,20.09,0.14
4,2024,7,4047365,Josh Jacobs,RB,GB,11.00,13.44,12.22,15.52,65.18,0.40,2.44,16.91,0.09


,year,week,player_id,player,position,team,points,points_ppr,points_half,receptions,reception_yards,reception_tds
0,2024,7,4258173,Nico Collins,WR,HOU,NaN,NaN,NaN,NaN,NaN,NaN
1,2024,7,4362628,Ja'Marr Chase,WR,CIN,11.49,17.53,14.51,6.04,82.43,0.56
2,2024,7,3116165,Chris Godwin,WR,TB,10.60,17.17,13.89,6.57,77.93,0.47
3,2024,7,4047650,DK Metcalf,WR,SEA,9.92,15.20,12.56,5.28,72.42,0.47
4,2024,7,4241389,CeeDee Lamb,WR,DAL,0.00,0.00,0.00,0.00,0.00,0.00


QB merged data: 66 rows
RB merged data: 233 rows
WR merged data: 390 rows


,year,week,player_id,player,position,team,points,points_ppr,points_half,passing_attempts,passing_completions,passing_yards,passing_tds,FirstName,LastName,Team,Position,This Week,Last Week,Difference
0,2024,7,15864,Geno Smith,QB,SEA,17.41,17.41,17.41,37.83,26.32,262.52,1.42,Geno,Smith,SEA,QB,"$7,600","$7,900",-300
1,2024,7,4361741,Brock Purdy,QB,SF,17.68,17.68,17.68,31.92,20.88,253.64,1.63,Brock,Purdy,SF,QB,"$7,700","$7,800",-100
2,2024,7,2577417,Dak Prescott,QB,DAL,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Dak,Prescott,NaN,NaN,NaN,NaN,NaN
3,2024,7,14880,Kirk Cousins,QB,ATL,16.76,16.76,16.76,34.13,22.74,260.32,1.71,Kirk,Cousins,ATL,QB,"$7,200","$7,400",-200
4,2024,7,3915511,Joe Burrow,QB,CIN,17.91,17.91,17.91,34.03,22.86,256.04,1.68,Joe,Burrow,CIN,QB,"$8,200","$8,100",+100


,year,week,player_id,player,position,team,points,points_ppr,points_half,rushing_attempts,...,receptions,reception_yards,reception_tds,FirstName,LastName,Team,Position,This Week,Last Week,Difference
0,2024,7,3043078,Derrick Henry,RB,BAL,15.73,16.96,16.34,19.31,...,1.22,10.17,0.05,Derrick,Henry,BAL,RB,"$9,300","$9,100",+200
1,2024,7,4360569,Jordan Mason,RB,SF,10.96,12.56,11.76,15.95,...,1.60,11.42,0.04,Jordan,Mason,SF,RB,"$8,800","$8,900",-100
2,2024,7,4241416,Chuba Hubbard,RB,CAR,11.91,15.04,13.47,16.63,...,3.14,22.85,0.10,Chuba,Hubbard,CAR,RB,"$7,400","$7,400",0
3,2024,7,3929630,Saquon Barkley,RB,PHI,14.49,17.25,15.87,19.14,...,2.76,20.09,0.14,Saquon,Barkley,PHI,RB,"$9,000","$9,200",-200
4,2024,7,4047365,Josh Jacobs,RB,GB,11.00,13.44,12.22,15.52,...,2.44,16.91,0.09,Josh,Jacobs,GB,RB,"$7,300","$7,600",-300


,year,week,player_id,player,position,team,points,points_ppr,points_half,receptions,reception_yards,reception_tds,FirstName,LastName,Team,Position,This Week,Last Week,Difference
0,2024,7,4258173,Nico Collins,WR,HOU,NaN,NaN,NaN,NaN,NaN,NaN,Nico,Collins,HOU,WR,"$4,000","$8,900","-4,900"
1,2024,7,4362628,Ja'Marr Chase,WR,CIN,11.49,17.53,14.51,6.04,82.43,0.56,Ja'Marr,Chase,NaN,NaN,NaN,NaN,NaN
2,2024,7,3116165,Chris Godwin,WR,TB,10.60,17.17,13.89,6.57,77.93,0.47,Chris,Godwin,TB,WR,"$7,900","$7,500",+400
3,2024,7,4047650,DK Metcalf,WR,SEA,9.92,15.20,12.56,5.28,72.42,0.47,DK,Metcalf,SEA,WR,"$8,100","$8,300",-200
4,2024,7,4241389,CeeDee Lamb,WR,DAL,0.00,0.00,0.00,0.00,0.00,0.00,CeeDee,Lamb,NaN,NaN,NaN,NaN,NaN


In [71]:
# modify_cols_merge_salary_and_projections
def modify_cols_merge_salary_and_projections(projections_df, salary_df):
    """
    This function merges salary and projection data based on FirstName and LastName,
    and reorders columns accordingly while removing unnecessary columns.
    """
    
    # Ensure salary_df has FirstName and LastName already
    if 'FirstName' not in salary_df.columns or 'LastName' not in salary_df.columns:
        print("Error: 'FirstName' and/or 'LastName' columns are missing in salary DataFrame.")
        print(f"Available columns in salary_df: {salary_df.columns}")
        return None
    
    # Ensure projections have player names split into firstName and lastName
    projections_df[['firstName', 'lastName']] = projections_df['player'].str.split(' ', n=1, expand=True)
    
    # Merge the two dataframes
    merged_df = pd.merge(projections_df, salary_df, left_on=['firstName', 'lastName'], right_on=['FirstName', 'LastName'], how='left')

    # Drop unwanted columns ('FirstName_x', 'LastName_x', 'FirstName_y', 'LastName_y', 'Team', 'Position')
    columns_to_drop = ['FirstName_x', 'LastName_x', 'FirstName_y', 'LastName_y', 'Team', 'Position']
    existing_columns_to_drop = [col for col in columns_to_drop if col in merged_df.columns]  # Only drop if they exist
    merged_df.drop(columns=existing_columns_to_drop, inplace=True)

    # Debugging: Check columns after dropping
    # print(f"Columns after dropping: {merged_df.columns}")

    # Reorder columns to move 'firstName' before 'lastName' right after 'player'
    cols = list(merged_df.columns)
    player_index = cols.index('player')
    first_last_name = ['firstName', 'lastName']
    
    # Insert 'firstName' before 'lastName'
    for name in first_last_name[::-1]:  # Reverse the order to insert correctly
        cols.insert(player_index + 1, cols.pop(cols.index(name)))
    
    merged_df = merged_df[cols]
    
    return merged_df

# Call the function with in-memory DataFrames
df_qb_merge2_projection_salary, df_rb_merge2_projection_salary, df_wr_merge2_projection_salary = merge2_salary_and_projections(
    df_qb_proj, df_rb_proj, df_wr_proj,  # Projections DataFrames
    df_qb_salary, df_rb_salary, df_wr_salary  # Salary DataFrames
)

Columns after dropping: Index(['year', 'week', 'player_id', 'player', 'position', 'team', 'points',
       'points_ppr', 'points_half', 'passing_attempts', 'passing_completions',
       'passing_yards', 'passing_tds', 'firstName', 'lastName', 'This Week',
       'Last Week', 'Difference'],
      dtype='object')
Columns after dropping: Index(['year', 'week', 'player_id', 'player', 'position', 'team', 'points',
       'points_ppr', 'points_half', 'rushing_attempts', 'rushing_yards',
       'rushing_tds', 'receptions', 'reception_yards', 'reception_tds',
       'firstName', 'lastName', 'This Week', 'Last Week', 'Difference'],
      dtype='object')
Columns after dropping: Index(['year', 'week', 'player_id', 'player', 'position', 'team', 'points',
       'points_ppr', 'points_half', 'receptions', 'reception_yards',
       'reception_tds', 'firstName', 'lastName', 'This Week', 'Last Week',
       'Difference'],
      dtype='object')


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,points_half,passing_attempts,passing_completions,passing_yards,passing_tds,This Week,Last Week,Difference
0,2024,7,15864,Geno Smith,Geno,Smith,QB,SEA,17.41,17.41,17.41,37.83,26.32,262.52,1.42,"$7,600","$7,900",-300
1,2024,7,4361741,Brock Purdy,Brock,Purdy,QB,SF,17.68,17.68,17.68,31.92,20.88,253.64,1.63,"$7,700","$7,800",-100
2,2024,7,2577417,Dak Prescott,Dak,Prescott,QB,DAL,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
3,2024,7,14880,Kirk Cousins,Kirk,Cousins,QB,ATL,16.76,16.76,16.76,34.13,22.74,260.32,1.71,"$7,200","$7,400",-200
4,2024,7,3915511,Joe Burrow,Joe,Burrow,QB,CIN,17.91,17.91,17.91,34.03,22.86,256.04,1.68,"$8,200","$8,100",+100


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,points_half,rushing_attempts,rushing_yards,rushing_tds,receptions,reception_yards,reception_tds,This Week,Last Week,Difference
0,2024,7,3043078,Derrick Henry,Derrick,Henry,RB,BAL,15.73,16.96,16.34,19.31,91.51,0.90,1.22,10.17,0.05,"$9,300","$9,100",+200
1,2024,7,4360569,Jordan Mason,Jordan,Mason,RB,SF,10.96,12.56,11.76,15.95,68.95,0.47,1.60,11.42,0.04,"$8,800","$8,900",-100
2,2024,7,4241416,Chuba Hubbard,Chuba,Hubbard,RB,CAR,11.91,15.04,13.47,16.63,69.46,0.38,3.14,22.85,0.10,"$7,400","$7,400",0
3,2024,7,3929630,Saquon Barkley,Saquon,Barkley,RB,PHI,14.49,17.25,15.87,19.14,85.08,0.56,2.76,20.09,0.14,"$9,000","$9,200",-200
4,2024,7,4047365,Josh Jacobs,Josh,Jacobs,RB,GB,11.00,13.44,12.22,15.52,65.18,0.40,2.44,16.91,0.09,"$7,300","$7,600",-300


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,points_half,receptions,reception_yards,reception_tds,This Week,Last Week,Difference
0,2024,7,4258173,Nico Collins,Nico,Collins,WR,HOU,NaN,NaN,NaN,NaN,NaN,NaN,"$4,000","$8,900","-4,900"
1,2024,7,4362628,Ja'Marr Chase,Ja'Marr,Chase,WR,CIN,11.49,17.53,14.51,6.04,82.43,0.56,NaN,NaN,NaN
2,2024,7,3116165,Chris Godwin,Chris,Godwin,WR,TB,10.60,17.17,13.89,6.57,77.93,0.47,"$7,900","$7,500",+400
3,2024,7,4047650,DK Metcalf,DK,Metcalf,WR,SEA,9.92,15.20,12.56,5.28,72.42,0.47,"$8,100","$8,300",-200
4,2024,7,4241389,CeeDee Lamb,CeeDee,Lamb,WR,DAL,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN


In [73]:
def scrape_rostered_percentage_over_weeks(positions, scoring='PPR', range_type='week'):
    current_week = get_current_week()
    
    # Dictionary to store DataFrames for each position
    rostered_dataframes = {}
    
    for position in positions:
        # Initialize an empty DataFrame to accumulate data
        rostered_df = pd.DataFrame()
        
        # Loop over weeks from current_week down to 1
        for week in range(current_week, 0, -1):
            # Construct the URL for the given position and week
            url = f"https://www.fantasypros.com/nfl/stats/{position}.php?week={week}&scoring={scoring}&range={range_type}"
            
            # Fetch the page content
            response = requests.get(url)
            
            # Check if the page was fetched successfully
            if response.status_code != 200:
                print(f"Failed to fetch the page for {position} week {week}. Status code: {response.status_code}")
                continue
            
            # Parse the page content using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Locate the table containing the stats data
            table = soup.find('table')
            
            # Extract the table headers
            headers = [header.text.strip() for header in table.find_all('th')]
            
            # Find the indices of the "Player" and "ROST" columns
            if "Player" not in headers or "ROST" not in headers:
                print(f"'Player' or 'ROST' column not found for {position} week {week}.")
                continue
            
            player_index = headers.index("Player")
            rost_index = headers.index("ROST")
            
            # Extract the table rows
            rows = []
            for row in table.find_all('tr')[1:]:  # Skip the header row
                cols = row.find_all('td')
                cols = [ele.get_text(strip=True) for ele in cols]  # Clean up whitespace
                
                # Only process rows with the expected number of columns
                if len(cols) >= rost_index + 1:
                    player_name = cols[player_index]  # Player name
                    rost_percentage = cols[rost_index]  # Rostered percentage
                    rows.append([player_name, rost_percentage])
            
            # Create a DataFrame for the current week
            week_df = pd.DataFrame(rows, columns=["Player", f"wk{week}"])
            
            # Convert the rostered percentage to float (remove '%' sign)
            week_df[f"wk{week}"] = week_df[f"wk{week}"].str.replace('%', '').astype(float)
            
            if rostered_df.empty:
                # First week's data, initialize the cumulative DataFrame
                rostered_df = week_df
            else:
                # Merge with the cumulative DataFrame
                rostered_df = pd.merge(rostered_df, week_df, on='Player', how='outer')
        
        # After looping through all weeks, sort columns in descending order
        # First, get the list of columns except 'Player'
        cols = rostered_df.columns.tolist()
        cols.remove('Player')
        # Sort the week columns in descending order
        cols_sorted = sorted(cols, key=lambda x: int(x[2:]), reverse=True)
        # Rearrange the columns
        rostered_df = rostered_df[['Player'] + cols_sorted]
        
        # Optionally, you can sort the DataFrame by current week's ROST
        if f"wk{current_week}" in rostered_df.columns:
            rostered_df = rostered_df.sort_values(by=f"wk{current_week}", ascending=False)
        
        # Store the DataFrame for the position
        rostered_dataframes[position] = rostered_df
        
        # Display the first few rows of the DataFrame for verification
        display(rostered_df.head())
    
    print("Scraping completed for all positions.")
    
    # Return the DataFrames for each position
    return rostered_dataframes

# Scrape rostered percentage over weeks for all positions
positions = ['qb', 'wr', 'rb', 'te', 'flex', 'dst']
rostered_dataframes = scrape_rostered_percentage_over_weeks(positions, scoring='PPR', range_type='week')

,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
73,Lamar Jackson(BAL),99.8,99.8,99.8,99.8,99.8,99.8,99.8
62,Josh Allen(BUF),99.8,99.8,99.8,99.8,99.8,99.8,99.8
47,Jalen Hurts(PHI),99.6,99.6,99.6,99.6,99.6,99.6,99.6
16,C.J. Stroud(HOU),97.9,97.9,97.9,97.9,97.9,97.9,97.9
87,Patrick Mahomes II(KC),97.8,97.8,97.8,97.8,97.8,97.8,97.8


,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
10,Amon-Ra St. Brown(DET),99.8,99.8,99.8,99.8,99.8,99.8,99.8
115,Ja'Marr Chase(CIN),99.8,99.8,99.8,99.8,99.8,99.8,99.8
168,Justin Jefferson(MIN),99.8,99.8,99.8,99.8,99.8,99.8,99.8
40,CeeDee Lamb(DAL),99.8,99.8,99.8,99.8,99.8,99.8,99.8
0,A.J. Brown(PHI),99.6,99.6,99.6,99.6,99.6,99.6,99.6


,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
85,Jahmyr Gibbs(DET),99.8,99.8,99.8,99.8,99.8,99.8,99.8
166,Saquon Barkley(PHI),99.8,99.8,99.8,99.8,99.8,99.8,99.8
14,Bijan Robinson(ATL),99.8,99.8,99.8,99.8,99.8,99.8,99.8
56,Derrick Henry(BAL),99.7,99.7,99.7,99.7,99.7,99.7,99.7
105,Jonathan Taylor(IND),99.7,99.7,99.7,99.7,99.7,99.7,99.7


,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
171,Travis Kelce(KC),99.5,99.5,99.5,99.5,99.5,99.5,99.5
66,George Kittle(SF),99.1,99.1,99.1,99.1,99.1,99.1,99.1
174,Trey McBride(ARI),98.1,98.1,98.1,98.1,98.1,98.1,98.1
151,Sam LaPorta(DET),97.6,97.6,97.6,97.6,97.6,97.6,97.6
20,Brock Bowers(LV),97.1,97.1,97.1,97.1,97.1,97.1,97.1


,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
73,Lamar Jackson(BAL),99.8,99.8,99.8,99.8,99.8,99.8,99.8
62,Josh Allen(BUF),99.8,99.8,99.8,99.8,99.8,99.8,99.8
47,Jalen Hurts(PHI),99.6,99.6,99.6,99.6,99.6,99.6,99.6
16,C.J. Stroud(HOU),97.9,97.9,97.9,97.9,97.9,97.9,97.9
87,Patrick Mahomes II(KC),97.8,97.8,97.8,97.8,97.8,97.8,97.8


,Player,wk7,wk6,wk5,wk4,wk3,wk2,wk1
27,San Francisco 49ers(SF),92.5,92.5,92.5,92.5,92.5,92.5,92.5
26,Pittsburgh Steelers(PIT),91.7,91.7,91.7,91.7,91.7,91.7,91.7
24,New York Jets(NYJ),89.2,89.2,89.2,89.2,89.2,89.2,89.2
3,Buffalo Bills(BUF),82.3,82.3,82.3,82.3,82.3,82.3,82.3
9,Denver Broncos(DEN),77.7,77.7,77.7,77.7,77.7,77.7,77.7


Scraping completed for all positions.


In [77]:
# test 1
# Assuming rostered_dataframes is the dictionary containing the rostered data scraped for each position
df_qb_merged_projections_salary_rostered = merge_with_rostered_data(df_qb_merge2_projection_salary, rostered_dataframes['qb'])
df_rb_merged_projections_salary_rostered = merge_with_rostered_data(df_rb_merge2_projection_salary, rostered_dataframes['rb'])
df_wr_merged_projections_salary_rostered = merge_with_rostered_data(df_wr_merge2_projection_salary, rostered_dataframes['wr'])

# Display the first few rows for each position
print("QB Merged Projections, Salary, and Rostered DataFrame:")
display(df_qb_merged_projections_salary_rostered.head())

print("\nRB Merged Projections, Salary, and Rostered DataFrame:")
display(df_rb_merged_projections_salary_rostered.head())

print("\nWR Merged Projections, Salary, and Rostered DataFrame:")
display(df_wr_merged_projections_salary_rostered.head())


QB Merged Projections, Salary, and Rostered DataFrame:


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,...,This Week Salary,Last Week Salary,Salary Differential,wk7 %rostered,wk6 %rostered,wk5 %rostered,wk4 %rostered,wk3 %rostered,wk2 %rostered,wk1 %rostered
0,2024,7,15864,Geno Smith,Geno,Smith,QB,SEA,17.41,17.41,...,"$7,600","$7,900",-300,58.8,58.8,58.8,58.8,58.8,58.8,58.8
1,2024,7,4361741,Brock Purdy,Brock,Purdy,QB,SF,17.68,17.68,...,"$7,700","$7,800",-100,91.2,91.2,91.2,91.2,91.2,91.2,91.2
2,2024,7,2577417,Dak Prescott,Dak,Prescott,QB,DAL,0.00,0.00,...,NaN,NaN,NaN,89.8,89.8,89.8,89.8,89.8,89.8,89.8
3,2024,7,14880,Kirk Cousins,Kirk,Cousins,QB,ATL,16.76,16.76,...,"$7,200","$7,400",-200,70.6,70.6,70.6,70.6,70.6,70.6,70.6
4,2024,7,3915511,Joe Burrow,Joe,Burrow,QB,CIN,17.91,17.91,...,"$8,200","$8,100",+100,96.7,96.7,96.7,96.7,96.7,96.7,96.7



RB Merged Projections, Salary, and Rostered DataFrame:


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,...,This Week Salary,Last Week Salary,Salary Differential,wk7 %rostered,wk6 %rostered,wk5 %rostered,wk4 %rostered,wk3 %rostered,wk2 %rostered,wk1 %rostered
0,2024,7,3043078,Derrick Henry,Derrick,Henry,RB,BAL,15.73,16.96,...,"$9,300","$9,100",+200,99.7,99.7,99.7,99.7,99.7,99.7,99.7
1,2024,7,4360569,Jordan Mason,Jordan,Mason,RB,SF,10.96,12.56,...,"$8,800","$8,900",-100,93.4,93.4,93.4,93.4,93.4,93.4,93.4
2,2024,7,4241416,Chuba Hubbard,Chuba,Hubbard,RB,CAR,11.91,15.04,...,"$7,400","$7,400",0,91.2,91.2,91.2,91.2,91.2,91.2,91.2
3,2024,7,3929630,Saquon Barkley,Saquon,Barkley,RB,PHI,14.49,17.25,...,"$9,000","$9,200",-200,99.8,99.8,99.8,99.8,99.8,99.8,99.8
4,2024,7,4047365,Josh Jacobs,Josh,Jacobs,RB,GB,11.00,13.44,...,"$7,300","$7,600",-300,98.7,98.7,98.7,98.7,98.7,98.7,98.7



WR Merged Projections, Salary, and Rostered DataFrame:


,year,week,player_id,player,firstName,lastName,position,team,points,points_ppr,...,This Week Salary,Last Week Salary,Salary Differential,wk7 %rostered,wk6 %rostered,wk5 %rostered,wk4 %rostered,wk3 %rostered,wk2 %rostered,wk1 %rostered
0,2024,7,4258173,Nico Collins,Nico,Collins,WR,HOU,NaN,NaN,...,"$4,000","$8,900","-4,900",98.5,98.5,98.5,98.5,98.5,98.5,98.5
1,2024,7,4362628,Ja'Marr Chase,Ja'Marr,Chase,WR,CIN,11.49,17.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024,7,3116165,Chris Godwin,Chris,Godwin,WR,TB,10.60,17.17,...,"$7,900","$7,500",+400,96.9,96.9,96.9,96.9,96.9,96.9,96.9
3,2024,7,4047650,DK Metcalf,DK,Metcalf,WR,SEA,9.92,15.20,...,"$8,100","$8,300",-200,98.3,98.3,98.3,98.3,98.3,98.3,98.3
4,2024,7,4241389,CeeDee Lamb,CeeDee,Lamb,WR,DAL,0.00,0.00,...,NaN,NaN,NaN,99.8,99.8,99.8,99.8,99.8,99.8,99.8


In [87]:
# test 1
# Scrape red zone stats for all weeks up to the current week
def scrape_red_zone_stats():
    current_week = get_current_week()  # Get the current week
    position_data = {}  # Dictionary to store data for each position
    current_week_data = {}  # Dictionary to store current week data for each position
    
    # Loop over positions
    for position, url_template in base_urls.items():
        all_weeks_data = pd.DataFrame()  # DataFrame to store data for all weeks
        
        # Loop over weeks from week 1 to current_week
        for week in range(1, current_week + 1):
            url = url_template.format(week=week)
            response = requests.get(url)
            
            if response.status_code != 200:
                print(f"Failed to fetch data for {position} week {week}. Status: {response.status_code}")
                continue
            
            # Parse the page content
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table')
            
            if not table:
                print(f"No table found for {position} week {week}.")
                continue
            
            # Extract the headers and data rows
            headers = [header.text.strip() for header in table.find_all('th')]
            rows = [[col.text.strip() for col in row.find_all('td')] for row in table.find_all('tr')[1:]]
            
            # Create a DataFrame for the current week
            week_df = pd.DataFrame(rows, columns=headers)
            week_df['Week'] = week  # Add a 'Week' column
            
            # Normalize column names by converting them to lowercase
            week_df.columns = week_df.columns.str.lower()
            
            # Distinguish columns for QB (passing and rushing stats)
            if position == 'qb':
                # Rename the overlapping columns for passing and rushing stats
                week_df = week_df.rename(columns={
                    'comp': 'pass_comp',
                    'att': 'pass_att', 
                    'yds': 'pass_yds',
                    'td': 'pass_td',
                    'int': 'pass_int',
                    'sacks': 'pass_sacks',
                    'att.1': 'rush_att',
                    'yds.1': 'rush_yds',
                    'td.1': 'rush_td'
                })
                selected_columns = ['player', 'week', 'pass_comp', 'pass_att', 'pass_yds', 'pass_td', 'pass_int', 'pass_sacks', 'rush_att', 'rush_yds', 'rush_td']
            else:
                # For RB, WR, TE: rename the columns to distinguish rushing and receiving
                week_df = week_df.rename(columns={
                    'att': 'rush_att',
                    'yds': 'rush_yds',
                    'td': 'rush_td',
                    'rec': 'rec_rec',
                    'tgt': 'rec_tgt',
                    'yds.1': 'rec_yds',
                    'td.1': 'rec_td'
                })
                selected_columns = ['player', 'week', 'rush_att', 'rush_yds', 'rush_td', 'rec_rec', 'rec_tgt', 'rec_yds', 'rec_td']
            
            # Filter the DataFrame to the selected columns
            selected_columns = [col for col in selected_columns if col in week_df.columns]  # Only keep existing columns
            week_df = week_df[selected_columns]  # Filter the DataFrame
            
            # Append to the cumulative DataFrame
            all_weeks_data = pd.concat([all_weeks_data, week_df], ignore_index=True)
        
        # Store the data for all weeks
        position_data[position] = all_weeks_data
        
        # Also store the data for the current week only
        current_week_data[position] = all_weeks_data[all_weeks_data['week'] == current_week]
    
    return position_data, current_week_data

# Example usage
position_data, current_week_data = scrape_red_zone_stats()

# Display the first few rows for each position (all weeks)
for position in position_data:
    print(f"\nAll weeks data for {position.upper()}:")
    display(position_data[position].head())



All weeks data for QB:


,player,week,pass_comp,pass_att,pass_att,pass_yds,pass_yds,pass_td,pass_td,pass_int,pass_sacks
0,None,1,None,None,None,None,None,None,None,None,None
1,Josh Allen (BUF),1,3,5,5,29,20,2,2,0,2
2,Baker Mayfield (TB),1,4,5,0,26,0,3,0,0,0
3,Jayden Daniels (WAS),1,1,1,3,10,2,0,2,0,0
4,Derek Carr (NO),1,2,2,0,19,0,2,0,0,0



All weeks data for RB:


,player,week,rush_att,rush_yds,rush_yds,rush_td,rush_td,rec_rec,rec_tgt
0,None,1,None,None,None,None,None,None,None
1,Saquon Barkley (PHI),1,8,33,18,2,1,1,1
2,Jordan Mason (SF),1,5,26,0,1,0,0,0
3,Kyren Williams (LAR),1,4,8,8,1,0,1,1
4,Joe Mixon (HOU),1,5,15,0,1,0,0,0



All weeks data for WR:


,player,week,rush_att,rush_yds,rush_yds,rush_td,rush_td,rec_rec,rec_tgt
0,None,1,None,None,None,None,None,None,None
1,Mike Evans (TB),1,0,18,0,2,0,2,3
2,Stefon Diggs (HOU),1,0,11,0,2,0,3,3
3,Brian Thomas Jr. (JAC),1,0,14,0,1,0,1,1
4,Cooper Kupp (LAR),1,1,13,1,1,0,2,3



All weeks data for TE:


,player,week,rush_att,rush_yds,rush_yds,rush_td,rush_td,rec_rec,rec_tgt
0,None,1,None,None,None,None,None,None,None
1,Chig Okonkwo (TEN),1,0,17,0,1,0,1,1
2,Juwan Johnson (NO),1,0,16,0,1,0,1,1
3,Kyle Pitts (ATL),1,0,12,0,1,0,1,1
4,Foster Moreau (NO),1,0,3,0,1,0,1,1


In [89]:
# test 2
# Scrape red zone stats for all weeks up to the current week
def scrape_red_zone_stats():
    current_week = get_current_week()  # Get the current week
    position_data = {}  # Dictionary to store data for each position
    current_week_data = {}  # Dictionary to store current week data for each position
    
    # Loop over positions
    for position, url_template in base_urls.items():
        all_weeks_data = pd.DataFrame()  # DataFrame to store data for all weeks
        
        # Loop over weeks from week 1 to current_week
        for week in range(1, current_week + 1):
            url = url_template.format(week=week)
            response = requests.get(url)
            
            if response.status_code != 200:
                print(f"Failed to fetch data for {position} week {week}. Status: {response.status_code}")
                continue
            
            # Parse the page content
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table')
            
            if not table:
                print(f"No table found for {position} week {week}.")
                continue
            
            # Extract the headers and data rows
            headers = [header.text.strip() for header in table.find_all('th')]
            rows = [[col.text.strip() for col in row.find_all('td')] for row in table.find_all('tr')[1:]]
            
            # Create a DataFrame for the current week
            week_df = pd.DataFrame(rows, columns=headers)
            week_df['Week'] = week  # Add a 'Week' column
            
            # Normalize column names by converting them to lowercase
            week_df.columns = week_df.columns.str.lower()
            
            # Split the 'Player' column into 'FirstName', 'LastName', and 'Team'
            week_df[['firstName', 'lastName', 'team']] = week_df['player'].str.extract(r'(\w+)\s+(\w+)\s+\((.*)\)')
            
            # Drop the original 'Player' column
            week_df.drop(columns=['player'], inplace=True)
            
            # Distinguish columns for QB (passing and rushing stats)
            if position == 'qb':
                # Rename the overlapping columns for passing and rushing stats
                week_df = week_df.rename(columns={
                    'comp': 'pass_comp',
                    'att': 'pass_att', 
                    'yds': 'pass_yds',
                    'td': 'pass_td',
                    'int': 'pass_int',
                    'sacks': 'pass_sacks',
                    'att.1': 'rush_att',
                    'yds.1': 'rush_yds',
                    'td.1': 'rush_td'
                })
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'pass_comp', 'pass_att', 'pass_yds', 'pass_td', 'pass_int', 'pass_sacks', 'rush_att', 'rush_yds', 'rush_td']
            else:
                # For RB, WR, TE: rename the columns to distinguish rushing and receiving
                week_df = week_df.rename(columns={
                    'att': 'rush_att',
                    'yds': 'rush_yds',
                    'td': 'rush_td',
                    'rec': 'rec_rec',
                    'tgt': 'rec_tgt',
                    'yds.1': 'rec_yds',
                    'td.1': 'rec_td'
                })
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'rush_att', 'rush_yds', 'rush_td', 'rec_rec', 'rec_tgt', 'rec_yds', 'rec_td']
            
            # Filter the DataFrame to the selected columns
            selected_columns = [col for col in selected_columns if col in week_df.columns]  # Only keep existing columns
            week_df = week_df[selected_columns]  # Filter the DataFrame
            
            # Append to the cumulative DataFrame
            all_weeks_data = pd.concat([all_weeks_data, week_df], ignore_index=True)
        
        # Store the data for all weeks
        position_data[position] = all_weeks_data
        
        # Also store the data for the current week only
        current_week_data[position] = all_weeks_data[all_weeks_data['week'] == current_week]
    
    return position_data, current_week_data

# Example usage
position_data, current_week_data = scrape_red_zone_stats()

# Display the first few rows for each position (all weeks)
for position in position_data:
    print(f"\nAll weeks data for {position.upper()}:")
    display(position_data[position].head())



All weeks data for QB:


,firstName,lastName,team,week,pass_comp,pass_att,pass_att,pass_yds,pass_yds,pass_td,pass_td,pass_int,pass_sacks
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None
1,Josh,Allen,BUF,1,3,5,5,29,20,2,2,0,2
2,Baker,Mayfield,TB,1,4,5,0,26,0,3,0,0,0
3,Jayden,Daniels,WAS,1,1,1,3,10,2,0,2,0,0
4,Derek,Carr,NO,1,2,2,0,19,0,2,0,0,0



All weeks data for RB:


,firstName,lastName,team,week,rush_att,rush_yds,rush_yds,rush_td,rush_td,rec_rec,rec_tgt
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None
1,Saquon,Barkley,PHI,1,8,33,18,2,1,1,1
2,Jordan,Mason,SF,1,5,26,0,1,0,0,0
3,Kyren,Williams,LAR,1,4,8,8,1,0,1,1
4,Joe,Mixon,HOU,1,5,15,0,1,0,0,0



All weeks data for WR:


,firstName,lastName,team,week,rush_att,rush_yds,rush_yds,rush_td,rush_td,rec_rec,rec_tgt
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None
1,Mike,Evans,TB,1,0,18,0,2,0,2,3
2,Stefon,Diggs,HOU,1,0,11,0,2,0,3,3
3,NaN,NaN,NaN,1,0,14,0,1,0,1,1
4,Cooper,Kupp,LAR,1,1,13,1,1,0,2,3



All weeks data for TE:


,firstName,lastName,team,week,rush_att,rush_yds,rush_yds,rush_td,rush_td,rec_rec,rec_tgt
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None
1,Chig,Okonkwo,TEN,1,0,17,0,1,0,1,1
2,Juwan,Johnson,NO,1,0,16,0,1,0,1,1
3,Kyle,Pitts,ATL,1,0,12,0,1,0,1,1
4,Foster,Moreau,NO,1,0,3,0,1,0,1,1


In [91]:
# test 3
# Scrape red zone stats for all weeks up to the current week
def scrape_red_zone_stats():
    current_week = get_current_week()  # Get the current week
    position_data = {}  # Dictionary to store data for each position
    current_week_data = {}  # Dictionary to store current week data for each position
    
    # Loop over positions
    for position, url_template in base_urls.items():
        all_weeks_data = pd.DataFrame()  # DataFrame to store data for all weeks
        
        # Loop over weeks from week 1 to current_week
        for week in range(1, current_week + 1):
            url = url_template.format(week=week)
            response = requests.get(url)
            
            if response.status_code != 200:
                print(f"Failed to fetch data for {position} week {week}. Status: {response.status_code}")
                continue
            
            # Parse the page content
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table')
            
            if not table:
                print(f"No table found for {position} week {week}.")
                continue
            
            # Extract the headers and data rows
            headers = [header.text.strip() for header in table.find_all('th')]
            rows = [[col.text.strip() for col in row.find_all('td')] for row in table.find_all('tr')[1:]]
            
            # Create a DataFrame for the current week
            week_df = pd.DataFrame(rows, columns=headers)
            week_df['Week'] = week  # Add a 'Week' column
            
            # Normalize column names by converting them to lowercase
            week_df.columns = week_df.columns.str.lower()
            
            # Split the 'Player' column into 'FirstName', 'LastName', and 'Team'
            week_df[['firstName', 'lastName', 'team']] = week_df['player'].str.extract(r'(\w+)\s+(\w+)\s+\((.*)\)')
            
            # Drop the original 'Player' column
            week_df.drop(columns=['player'], inplace=True)
            
            if position == 'qb':
                # Rename columns for passing stats and drop unnecessary columns
                week_df = week_df.rename(columns={
                    'comp': 'pass_comp',
                    'att': 'pass_att', 
                    'pct': 'pass_pct',
                    'yds': 'pass_yds',
                    'y/a': 'pass_y/a',
                    'td': 'pass_td'
                })
                # Select the final columns
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'pass_comp', 'pass_att', 'pass_pct', 'pass_yds', 'pass_y/a', 'pass_td']
            
            elif position == 'rb':
                # Rename rushing and receiving columns
                week_df = week_df.rename(columns={
                    'att': 'rush_att',
                    'yds': 'rush_yds',
                    'y/a': 'rush_y/a',
                    'td': 'rush_td',
                    'pct': 'rush_pct',
                    'rec': 'rec_rec',
                    'tgt': 'rec_tgt',
                    'rec pct': 'rec_pct',
                    'yds.1': 'rec_yds',
                    'y/r': 'rec_y/r',
                    'td.1': 'rec_td',
                    'tgt pct': 'tgt_pct'
                })
                # Select the final columns
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'rush_att', 'rush_yds', 'rush_y/a', 'rush_td', 'rush_pct', 
                                    'rec_rec', 'rec_tgt', 'rec_pct', 'rec_yds', 'rec_y/r', 'rec_td', 'tgt_pct']
            
            elif position in ['wr', 'te']:
                # Rename receiving columns and remove rushing stats
                week_df = week_df.rename(columns={
                    'rec': 'rec_rec',
                    'tgt': 'rec_tgt',
                    'rec pct': 'rec_pct',
                    'yds': 'rec_yds',
                    'y/r': 'rec_y/r',
                    'td': 'rec_td',
                    'tgt pct': 'rec_tgt_pct'
                })
                # Select the final columns
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'rec_rec', 'rec_tgt', 'rec_pct', 'rec_yds', 'rec_y/r', 'rec_td', 'rec_tgt_pct']
            
            # Filter the DataFrame to the selected columns
            selected_columns = [col for col in selected_columns if col in week_df.columns]  # Only keep existing columns
            week_df = week_df[selected_columns]  # Filter the DataFrame
            
            # Append to the cumulative DataFrame
            all_weeks_data = pd.concat([all_weeks_data, week_df], ignore_index=True)
        
        # Store the data for all weeks
        position_data[position] = all_weeks_data
        
        # Also store the data for the current week only
        current_week_data[position] = all_weeks_data[all_weeks_data['week'] == current_week]
    
    return position_data, current_week_data

# Example usage
position_data, current_week_data = scrape_red_zone_stats()

# Display the first few rows for each position (all weeks)
for position in position_data:
    print(f"\nAll weeks data for {position.upper()}:")
    display(position_data[position].head())



All weeks data for QB:


,firstName,lastName,team,week,pass_comp,pass_att,pass_att,pass_pct,pass_pct,pass_yds,pass_yds,pass_y/a,pass_td,pass_td
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None,None
1,Josh,Allen,BUF,1,3,5,5,60.0%,100.0%,29,20,5.8,2,2
2,Baker,Mayfield,TB,1,4,5,0,80.0%,0%,26,0,5.2,3,0
3,Jayden,Daniels,WAS,1,1,1,3,100.0%,100.0%,10,2,10.0,0,2
4,Derek,Carr,NO,1,2,2,0,100.0%,0%,19,0,9.5,2,0



All weeks data for RB:


,firstName,lastName,team,week,rush_att,rush_yds,rush_yds,rush_y/a,rush_td,rush_td,rush_pct,rec_rec,rec_tgt,rec_pct,rec_y/r,tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None,None,None,None
1,Saquon,Barkley,PHI,1,8,33,18,4.1,2,1,100.0%,1,1,100.0%,18.0,100.0%
2,Jordan,Mason,SF,1,5,26,0,5.2,1,0,100.0%,0,0,0%,0,0%
3,Kyren,Williams,LAR,1,4,8,8,2.0,1,0,100.0%,1,1,100.0%,8.0,100.0%
4,Joe,Mixon,HOU,1,5,15,0,3.0,1,0,83.3%,0,0,0%,0,0%



All weeks data for WR:


,firstName,lastName,team,week,rec_rec,rec_tgt,rec_pct,rec_yds,rec_yds,rec_y/r,rec_td,rec_td,rec_tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None
1,Mike,Evans,TB,1,2,3,66.7%,18,0,9.0,2,0,75.0%
2,Stefon,Diggs,HOU,1,3,3,100.0%,11,0,3.7,2,0,75.0%
3,NaN,NaN,NaN,1,1,1,100.0%,14,0,14.0,1,0,100.0%
4,Cooper,Kupp,LAR,1,2,3,66.7%,13,1,6.5,1,0,60.0%



All weeks data for TE:


,firstName,lastName,team,week,rec_rec,rec_tgt,rec_pct,rec_yds,rec_yds,rec_y/r,rec_td,rec_td,rec_tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None
1,Chig,Okonkwo,TEN,1,1,1,100.0%,17,0,17.0,1,0,100.0%
2,Juwan,Johnson,NO,1,1,1,100.0%,16,0,16.0,1,0,50.0%
3,Kyle,Pitts,ATL,1,1,1,100.0%,12,0,12.0,1,0,100.0%
4,Foster,Moreau,NO,1,1,1,100.0%,3,0,3.0,1,0,50.0%


In [93]:
# test 3
# Scrape red zone stats for all weeks up to the current week
def scrape_red_zone_stats():
    current_week = get_current_week()  # Get the current week
    position_data = {}  # Dictionary to store data for each position
    current_week_data = {}  # Dictionary to store current week data for each position
    
    # Loop over positions
    for position, url_template in base_urls.items():
        all_weeks_data = pd.DataFrame()  # DataFrame to store data for all weeks
        
        # Loop over weeks from week 1 to current_week
        for week in range(1, current_week + 1):
            url = url_template.format(week=week)
            response = requests.get(url)
            
            if response.status_code != 200:
                print(f"Failed to fetch data for {position} week {week}. Status: {response.status_code}")
                continue
            
            # Parse the page content
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table')
            
            if not table:
                print(f"No table found for {position} week {week}.")
                continue
            
            # Extract the headers and data rows
            headers = [header.text.strip() for header in table.find_all('th')]
            rows = [[col.text.strip() for col in row.find_all('td')] for row in table.find_all('tr')[1:]]
            
            # Create a DataFrame for the current week
            week_df = pd.DataFrame(rows, columns=headers)
            week_df['Week'] = week  # Add a 'Week' column
            
            # Normalize column names by converting them to lowercase
            week_df.columns = week_df.columns.str.lower()
            
            # Split the 'Player' column into 'FirstName', 'LastName', and 'Team'
            week_df[['firstName', 'lastName', 'team']] = week_df['player'].str.extract(r'(\w+)\s+(\w+)\s+\((.*)\)')
            
            # Drop the original 'Player' column
            week_df.drop(columns=['player'], inplace=True)
            
            if position == 'qb':
                # Rename columns for passing stats and drop unnecessary columns
                week_df = week_df.rename(columns={
                    'comp': 'pass_comp',
                    'att': 'pass_att', 
                    'pct': 'pass_pct',
                    'yds': 'pass_yds',
                    'y/a': 'pass_y/a',
                    'td': 'pass_td'
                })
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'pass_comp', 'pass_att', 'pass_pct', 'pass_yds', 'pass_y/a', 'pass_td']
            
            elif position == 'rb':
                # Rename rushing and receiving columns
                week_df = week_df.rename(columns={
                    'att': 'rush_att',
                    'yds': 'rush_yds',
                    'y/a': 'rush_y/a',
                    'td': 'rush_td',
                    'pct': 'rush_pct',
                    'rec': 'rec_rec',
                    'tgt': 'rec_tgt',
                    'rec pct': 'rec_pct',
                    'yds.1': 'rec_yds',
                    'y/r': 'rec_y/r',
                    'td.1': 'rec_td',
                    'tgt pct': 'tgt_pct'
                })
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'rush_att', 'rush_yds', 'rush_y/a', 'rush_td', 'rush_pct', 
                                    'rec_rec', 'rec_tgt', 'rec_pct', 'rec_yds', 'rec_y/r', 'rec_td', 'tgt_pct']
            
            elif position in ['wr', 'te']:
                # Rename receiving columns and remove rushing stats
                week_df = week_df.rename(columns={
                    'rec': 'rec_rec',
                    'tgt': 'rec_tgt',
                    'rec pct': 'rec_pct',
                    'yds': 'rec_yds',
                    'y/r': 'rec_y/r',
                    'td': 'rec_td',
                    'tgt pct': 'rec_tgt_pct'
                })
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'rec_rec', 'rec_tgt', 'rec_pct', 'rec_yds', 'rec_y/r', 'rec_td', 'rec_tgt_pct']
            
            # Filter the DataFrame to the selected columns
            selected_columns = [col for col in selected_columns if col in week_df.columns]  # Only keep existing columns
            week_df = week_df[selected_columns]  # Filter the DataFrame

            # Add 'red_zone_' prefix to all columns except the first four columns
            week_df.columns = ['firstName', 'lastName', 'team', 'week'] + [f"red_zone_{col}" for col in week_df.columns[4:]]
            
            # Append to the cumulative DataFrame
            all_weeks_data = pd.concat([all_weeks_data, week_df], ignore_index=True)
        
        # Store the data for all weeks
        position_data[position] = all_weeks_data
        
        # Also store the data for the current week only
        current_week_data[position] = all_weeks_data[all_weeks_data['week'] == current_week]
    
    return position_data, current_week_data

# Example usage
position_data, current_week_data = scrape_red_zone_stats()

# Display the first few rows for each position (all weeks)
for position in position_data:
    print(f"\nAll weeks data for {position.upper()}:")
    display(position_data[position].head())



All weeks data for QB:


,firstName,lastName,team,week,red_zone_pass_comp,red_zone_pass_att,red_zone_pass_att,red_zone_pass_pct,red_zone_pass_pct,red_zone_pass_yds,red_zone_pass_yds,red_zone_pass_y/a,red_zone_pass_td,red_zone_pass_td
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None,None
1,Josh,Allen,BUF,1,3,5,5,60.0%,100.0%,29,20,5.8,2,2
2,Baker,Mayfield,TB,1,4,5,0,80.0%,0%,26,0,5.2,3,0
3,Jayden,Daniels,WAS,1,1,1,3,100.0%,100.0%,10,2,10.0,0,2
4,Derek,Carr,NO,1,2,2,0,100.0%,0%,19,0,9.5,2,0



All weeks data for RB:


,firstName,lastName,team,week,red_zone_rush_att,red_zone_rush_yds,red_zone_rush_yds,red_zone_rush_y/a,red_zone_rush_td,red_zone_rush_td,red_zone_rush_pct,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_y/r,red_zone_tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None,None,None,None
1,Saquon,Barkley,PHI,1,8,33,18,4.1,2,1,100.0%,1,1,100.0%,18.0,100.0%
2,Jordan,Mason,SF,1,5,26,0,5.2,1,0,100.0%,0,0,0%,0,0%
3,Kyren,Williams,LAR,1,4,8,8,2.0,1,0,100.0%,1,1,100.0%,8.0,100.0%
4,Joe,Mixon,HOU,1,5,15,0,3.0,1,0,83.3%,0,0,0%,0,0%



All weeks data for WR:


,firstName,lastName,team,week,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_yds,red_zone_rec_yds,red_zone_rec_y/r,red_zone_rec_td,red_zone_rec_td,red_zone_rec_tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None
1,Mike,Evans,TB,1,2,3,66.7%,18,0,9.0,2,0,75.0%
2,Stefon,Diggs,HOU,1,3,3,100.0%,11,0,3.7,2,0,75.0%
3,NaN,NaN,NaN,1,1,1,100.0%,14,0,14.0,1,0,100.0%
4,Cooper,Kupp,LAR,1,2,3,66.7%,13,1,6.5,1,0,60.0%



All weeks data for TE:


,firstName,lastName,team,week,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_yds,red_zone_rec_yds,red_zone_rec_y/r,red_zone_rec_td,red_zone_rec_td,red_zone_rec_tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None
1,Chig,Okonkwo,TEN,1,1,1,100.0%,17,0,17.0,1,0,100.0%
2,Juwan,Johnson,NO,1,1,1,100.0%,16,0,16.0,1,0,50.0%
3,Kyle,Pitts,ATL,1,1,1,100.0%,12,0,12.0,1,0,100.0%
4,Foster,Moreau,NO,1,1,1,100.0%,3,0,3.0,1,0,50.0%


In [95]:
# test 4
# Scrape red zone stats for all weeks up to the current week
def scrape_red_zone_stats():
    current_week = get_current_week()  # Get the current week
    position_data = {}  # Dictionary to store data for each position
    current_week_data = {}  # Dictionary to store current week data for each position
    
    # Loop over positions
    for position, url_template in base_urls.items():
        all_weeks_data = pd.DataFrame()  # DataFrame to store data for all weeks
        
        # Loop over weeks from week 1 to current_week
        for week in range(1, current_week + 1):
            url = url_template.format(week=week)
            response = requests.get(url)
            
            if response.status_code != 200:
                print(f"Failed to fetch data for {position} week {week}. Status: {response.status_code}")
                continue
            
            # Parse the page content
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table')
            
            if not table:
                print(f"No table found for {position} week {week}.")
                continue
            
            # Extract the headers and data rows
            headers = [header.text.strip() for header in table.find_all('th')]
            rows = [[col.text.strip() for col in row.find_all('td')] for row in table.find_all('tr')[1:]]
            
            # Create a DataFrame for the current week
            week_df = pd.DataFrame(rows, columns=headers)
            week_df['Week'] = week  # Add a 'Week' column
            
            # Normalize column names by converting them to lowercase
            week_df.columns = week_df.columns.str.lower()
            
            # Split the 'Player' column into 'FirstName', 'LastName', and 'Team'
            week_df[['firstName', 'lastName', 'team']] = week_df['player'].str.extract(r'(\w+)\s+(\w+)\s+\((.*)\)')
            
            # Drop the original 'Player' column
            week_df.drop(columns=['player'], inplace=True)
            
            if position == 'qb':
                # Rename columns for passing stats and drop unnecessary columns
                week_df = week_df.rename(columns={
                    'comp': 'pass_comp',
                    'att': 'pass_att', 
                    'pct': 'pass_pct',
                    'yds': 'pass_yds',
                    'y/a': 'pass_y/a',
                    'td': 'pass_td'
                }).drop_duplicates(subset=['pass_att', 'pass_yds', 'pass_td'])  # Drop duplicate columns
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'pass_comp', 'pass_att', 'pass_pct', 'pass_yds', 'pass_y/a', 'pass_td']
            
            elif position == 'rb':
                # Rename rushing and receiving columns and drop duplicates
                week_df = week_df.rename(columns={
                    'att': 'rush_att',
                    'yds': 'rush_yds',
                    'y/a': 'rush_y/a',
                    'td': 'rush_td',
                    'pct': 'rush_pct',
                    'rec': 'rec_rec',
                    'tgt': 'rec_tgt',
                    'rec pct': 'rec_pct',
                    'yds.1': 'rec_yds',
                    'y/r': 'rec_y/r',
                    'td.1': 'rec_td',
                    'tgt pct': 'tgt_pct'
                }).drop_duplicates(subset=['rush_yds', 'rush_td'])  # Drop duplicate columns
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'rush_att', 'rush_yds', 'rush_y/a', 'rush_td', 'rush_pct', 
                                    'rec_rec', 'rec_tgt', 'rec_pct', 'rec_yds', 'rec_y/r', 'rec_td', 'tgt_pct']
            
            elif position in ['wr', 'te']:
                # Rename receiving columns and remove rushing stats and drop duplicates
                week_df = week_df.rename(columns={
                    'rec': 'rec_rec',
                    'tgt': 'rec_tgt',
                    'rec pct': 'rec_pct',
                    'yds': 'rec_yds',
                    'y/r': 'rec_y/r',
                    'td': 'rec_td',
                    'tgt pct': 'rec_tgt_pct'
                }).drop_duplicates(subset=['rec_yds', 'rec_td'])  # Drop duplicate columns
                selected_columns = ['firstName', 'lastName', 'team', 'week', 'rec_rec', 'rec_tgt', 'rec_pct', 'rec_yds', 'rec_y/r', 'rec_td', 'rec_tgt_pct']
            
            # Filter the DataFrame to the selected columns
            selected_columns = [col for col in selected_columns if col in week_df.columns]  # Only keep existing columns
            week_df = week_df[selected_columns]  # Filter the DataFrame

            # Add 'red_zone_' prefix to all columns except the first four columns
            week_df.columns = ['firstName', 'lastName', 'team', 'week'] + [f"red_zone_{col}" for col in week_df.columns[4:]]
            
            # Append to the cumulative DataFrame
            all_weeks_data = pd.concat([all_weeks_data, week_df], ignore_index=True)
        
        # Store the data for all weeks
        position_data[position] = all_weeks_data
        
        # Also store the data for the current week only
        current_week_data[position] = all_weeks_data[all_weeks_data['week'] == current_week]
    
    return position_data, current_week_data

# Example usage
position_data, current_week_data = scrape_red_zone_stats()

# Display the first few rows for each position (all weeks)
for position in position_data:
    print(f"\nAll weeks data for {position.upper()}:")
    display(position_data[position].head())



All weeks data for QB:


,firstName,lastName,team,week,red_zone_pass_comp,red_zone_pass_att,red_zone_pass_att,red_zone_pass_pct,red_zone_pass_pct,red_zone_pass_yds,red_zone_pass_yds,red_zone_pass_y/a,red_zone_pass_td,red_zone_pass_td
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None,None
1,Josh,Allen,BUF,1,3,5,5,60.0%,100.0%,29,20,5.8,2,2
2,Baker,Mayfield,TB,1,4,5,0,80.0%,0%,26,0,5.2,3,0
3,Jayden,Daniels,WAS,1,1,1,3,100.0%,100.0%,10,2,10.0,0,2
4,Derek,Carr,NO,1,2,2,0,100.0%,0%,19,0,9.5,2,0



All weeks data for RB:


,firstName,lastName,team,week,red_zone_rush_att,red_zone_rush_yds,red_zone_rush_yds,red_zone_rush_y/a,red_zone_rush_td,red_zone_rush_td,red_zone_rush_pct,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_y/r,red_zone_tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None,None,None,None
1,Saquon,Barkley,PHI,1,8,33,18,4.1,2,1,100.0%,1,1,100.0%,18.0,100.0%
2,Jordan,Mason,SF,1,5,26,0,5.2,1,0,100.0%,0,0,0%,0,0%
3,Kyren,Williams,LAR,1,4,8,8,2.0,1,0,100.0%,1,1,100.0%,8.0,100.0%
4,Joe,Mixon,HOU,1,5,15,0,3.0,1,0,83.3%,0,0,0%,0,0%



All weeks data for WR:


,firstName,lastName,team,week,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_yds,red_zone_rec_yds,red_zone_rec_y/r,red_zone_rec_td,red_zone_rec_td,red_zone_rec_tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None
1,Mike,Evans,TB,1,2,3,66.7%,18,0,9.0,2,0,75.0%
2,Stefon,Diggs,HOU,1,3,3,100.0%,11,0,3.7,2,0,75.0%
3,NaN,NaN,NaN,1,1,1,100.0%,14,0,14.0,1,0,100.0%
4,Cooper,Kupp,LAR,1,2,3,66.7%,13,1,6.5,1,0,60.0%



All weeks data for TE:


,firstName,lastName,team,week,red_zone_rec_rec,red_zone_rec_tgt,red_zone_rec_pct,red_zone_rec_yds,red_zone_rec_yds,red_zone_rec_y/r,red_zone_rec_td,red_zone_rec_td,red_zone_rec_tgt_pct
0,NaN,NaN,NaN,1,None,None,None,None,None,None,None,None,None
1,Chig,Okonkwo,TEN,1,1,1,100.0%,17,0,17.0,1,0,100.0%
2,Juwan,Johnson,NO,1,1,1,100.0%,16,0,16.0,1,0,50.0%
3,Kyle,Pitts,ATL,1,1,1,100.0%,12,0,12.0,1,0,100.0%
4,Foster,Moreau,NO,1,1,1,100.0%,3,0,3.0,1,0,50.0%
